In [3]:
import pandas as pd

In [4]:
# read data
df_data = pd.read_csv("data/stateUER.csv")
df_state = pd.read_csv("data/statesConversion.csv", dtype=str)

In [8]:
df_series = df_data.pivot(index="Label", columns="Series ID", values="Value")
# Drop Puerto and Seasonal adjusted Calfonia
df_series = df_series.drop(['LASST720000000000003','LAUST060000000000003'], axis=1)
df_series.columns = df_series.columns.str[5:7]

In [193]:
# rename Series ID to state name
state_code_to_name = dict(zip(df_state["state_code"], df_state["state_name"]))
df_series.columns = [state_code_to_name.get(col, col) for col in df_series.columns]
df_series = df_series.apply(pd.to_numeric)
df_series.index = pd.to_datetime(df_series.index)

In [197]:
df_series.sort_index().to_excel("data/cleanStateUER.xlsx")

In [12]:
df_data

,Series ID,Year,Period,Label,Value
0,LASST010000000000003,1980,M01,1980 Jan,7.6
1,LASST010000000000003,1980,M02,1980 Feb,7.7
2,LASST010000000000003,1980,M03,1980 Mar,8.0
3,LASST010000000000003,1980,M04,1980 Apr,8.3
4,LASST010000000000003,1980,M05,1980 May,8.7
5,LASST010000000000003,1980,M06,1980 Jun,9.0
6,LASST010000000000003,1980,M07,1980 Jul,9.2
7,LASST010000000000003,1980,M08,1980 Aug,9.3
8,LASST010000000000003,1980,M09,1980 Sep,9.4
9,LASST010000000000003,1980,M10,1980 Oct,9.5


In [11]:
df_series

Series ID,01,02,04,05,06,08,09,10,12,13,...,46,47,48,49,50,51,53,54,55,56
Label,,,,,,,,,,,,,,,,,,,,,
1980 Apr,8.3,9.5,6.6,7.1,6.7,5.4,5.5,7.2,5.9,6.3,...,4.5,6.7,5.3,6.0,5.7,5.0,7.6,9.5,6.8,3.5
1980 Aug,9.3,10.0,7.3,8.1,7.3,6.7,5.9,8.0,6.2,6.8,...,5.1,7.7,5.4,6.7,6.5,5.4,8.4,11.0,8.5,4.1
1980 Dec,9.6,9.3,6.2,8.2,7.1,6.2,5.9,7.7,6.2,6.1,...,5.2,8.0,5.3,6.7,6.0,5.2,8.4,11.0,8.0,4.5
1980 Feb,7.7,8.8,5.7,6.4,6.3,4.6,5.2,6.9,5.6,5.6,...,3.9,6.2,4.9,5.4,5.1,4.6,7.0,8.6,5.6,3.2
1980 Jan,7.6,8.7,5.4,6.2,6.2,4.5,5.1,6.8,5.6,5.4,...,3.7,6.1,4.7,5.2,5.0,4.5,6.8,8.2,5.2,3.1
1980 Jul,9.2,10.1,7.4,8.0,7.2,6.6,5.9,7.9,6.1,6.9,...,5.1,7.6,5.4,6.6,6.4,5.5,8.4,10.8,7.9,3.9
1980 Jun,9.0,10.0,7.2,7.8,7.1,6.4,5.8,7.8,6.1,6.8,...,5.0,7.4,5.4,6.4,6.3,5.4,8.2,10.5,7.7,3.7
1980 Mar,8.0,9.1,6.1,6.8,6.5,5.0,5.3,7.0,5.7,6.0,...,4.1,6.4,5.1,5.7,5.4,4.8,7.2,9.0,6.2,3.4
1980 May,8.7,9.8,7.0,7.5,6.9,5.9,5.7,7.5,6.0,6.6,...,4.7,7.1,5.4,6.2,6.0,5.2,7.9,10.0,7.3,3.6
